<a href="https://colab.research.google.com/github/hihohub/umbrella-networks/blob/main/joint_probability_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JOINT PROBABILITY NETWORK
- using just one folder structure (hierarchical for an umbrella network), the program also makes a general softmax model (levels = 0) from the bottom level folders, then combines the predictions of the general model with the predictions of the top level umbrella model
- their predictions are combined with joint probability multiplication

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
! wget https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py

--2022-04-16 04:35:23--  https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48184 (47K) [text/plain]
Saving to: ‘umbrella.py’

umbrella.py         100%[===================>]  47.05K  --.-KB/s    in 0.004s  

2022-04-16 04:35:23 (11.2 MB/s) - ‘umbrella.py’ saved [48184/48184]



In [3]:
from umbrella import Umbrella_Network,Umbrella_Label,Umbrella_Node,Joint_Probability_Network

In [4]:
import os
import sys
import numpy as np
import math
import random
from PIL import Image, ImageOps, ImageChops
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, add, BatchNormalization, Dropout, AveragePooling2D, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras import backend as backend
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
import copy
import time

In [6]:
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models"
target_image_size = 30
channels = 3
valid_ratio = 0.1 # validation set
test_ratio = 0.0 # test set
Model = Joint_Probability_Network(images_folder,serial_file,models_folder,target_image_size,channels,valid_ratio,test_ratio)

In [7]:
Model.traverse_training_data()

specialist
small_hierarchy trainX 154 (154,) trainY 154 (154, 3)
   animals trainX 0 (0,) trainY 0 (0,)
      alligator trainX 0 (0,) trainY 0 (0,)
      mammals trainX 0 (0,) trainY 0 (0,)
      salamanders trainX 0 (0,) trainY 0 (0,)
      snake trainX 0 (0,) trainY 0 (0,)
   objects trainX 0 (0,) trainY 0 (0,)
      automobiles trainX 0 (0,) trainY 0 (0,)
      bicycles trainX 0 (0,) trainY 0 (0,)
      boats trainX 0 (0,) trainY 0 (0,)
   plants trainX 0 (0,) trainY 0 (0,)
      cactus trainX 0 (0,) trainY 0 (0,)
      hedges trainX 0 (0,) trainY 0 (0,)
      shrubs trainX 0 (0,) trainY 0 (0,)
generalist
small_hierarchy trainX 154 (154,) trainY 154 (154, 10)
   alligator trainX 0 (0,) trainY 0 (0,)
   mammals trainX 0 (0,) trainY 0 (0,)
   salamanders trainX 0 (0,) trainY 0 (0,)
   snake trainX 0 (0,) trainY 0 (0,)
   automobiles trainX 0 (0,) trainY 0 (0,)
   bicycles trainX 0 (0,) trainY 0 (0,)
   boats trainX 0 (0,) trainY 0 (0,)
   cactus trainX 0 (0,) trainY 0 (0,)
   hedges t

In [8]:
Model.make_models()

making models for specialist
making model for small_hierarchy
making modes for generalist
making model for small_hierarchy


In [9]:
Model.train_models(eps=1,batch=10,aug=False,vrb=True)

training specialist
training model for small_hierarchy
trainX 154
trainY 154
16/16 [==============================] - 9s 7ms/step - loss: 1.0546 - accuracy: 0.4351
training generalist
training model for small_hierarchy
trainX 154
trainY 154
16/16 [==============================] - 1s 4ms/step - loss: 2.3230 - accuracy: 0.0844


In [10]:
Model.predict_validation_set()

validation accuracy
specialist
expected [[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]]
predicted [[0.2528699033321078, 0.2009342301461648, 0.5461958665217274], [0.2902808659120805, 0.1908728726950872, 0.5188462613928322], [0.23454495786614818, 0.20586256702630762, 0.5595924751075442], [0.2831747559081898, 0.1927840000199451, 0.5240412440718649], [0.20883401628835843, 0.2234962720708766, 0.567669711640765], [0.24576939295279956, 0.2028438515000284, 0.5513867555471722], [0.3403311746888706, 0.17741227151265015, 0.4822565537984792], [0.17430625594690852, 0.47381352812721284, 0.35188021592587865], [0.19065302610788143, 0.518248656409782, 0.2910983174823366], [0.1852089647392231, 0.5034501633183422, 0.3113408719424347], [0.18193474066582613, 0.49454989951732403, 0.32351535981685], [0.27031596087584797, 0.19624226263304853, 0.5334417764911034], [0.24304